In [ ]:
from langchain_experimental.tools import PythonREPLTool
from langgraph.prebuilt import create_react_agent
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain_openai import AzureChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from state import AgentState
from langgraph.types import Command
from langchain_core.messages import HumanMessage

token_provider = get_bearer_token_provider(DefaultAzureCredential(),"https://cognitiveservices.azure.com/.default")
model= AzureChatOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider= token_provider,
    azure_deployment= "gpt-4o-mini",
    model= "gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Create the agent
memory = MemorySaver()

@tool
def python_run_tool(code: str) -> str:
    """ This tool will help to excute the python code and print the chart"""
    code_tool = PythonREPLTool()
    result = code_tool.run(code)
    result_str = f"Successfully executed: python\n{code}\n\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

code_agent = create_react_agent(
    model=model,
    checkpointer=memory,
    tools=[python_run_tool],
    prompt="You are a helpful agent going to help print the chart, If the code is not going to print the chat please get the code"
    )

def chart_node(state: AgentState) -> AgentState:
    result= code_agent.invoke(state)
    state.messages[-1]=HumanMessage(
        content=result["messages"][-1].content, name="chart_node"
    )
    return Command(goto="search_node",update={"messages" : state})
